In [1]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, FloatType
from pyspark.sql.functions import *
import pandas as pd
import matplotlib.pyplot as plt
import time


In [2]:
import os
import sys
os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable
# Creating a spark context class
sc = SparkContext()

# Creating a spark session
spark = SparkSession.builder.appName("pyspark-notebook").master("spark://spark-master:7077").config("spark.executor.memory", "1024m").getOrCreate()

In [3]:
file_path = './data/salaries.csv'

data_engineer_salary = pd.read_csv(file_path)
styled_salaries = data_engineer_salary.head(10).style.set_properties(**{'text-align': 'left'}) 
styled_salaries.set_table_styles([{'selector': 'th', 'props': [('text-align', 'left')]}])
display(styled_salaries)

,work_year,experience_level,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,company_size
0,2024,SE,FT,AI Engineer,202730,USD,202730,US,0,US,M
1,2024,SE,FT,AI Engineer,92118,USD,92118,US,0,US,M
2,2024,SE,FT,Data Engineer,130500,USD,130500,US,0,US,M
3,2024,SE,FT,Data Engineer,96000,USD,96000,US,0,US,M
4,2024,SE,FT,Machine Learning Engineer,190000,USD,190000,US,0,US,M
5,2024,SE,FT,Machine Learning Engineer,160000,USD,160000,US,0,US,M
6,2024,MI,FT,ML Engineer,400000,USD,400000,US,0,US,M
7,2024,MI,FT,ML Engineer,65000,USD,65000,US,0,US,M
8,2024,EN,FT,Data Analyst,101520,USD,101520,US,0,US,M
9,2024,EN,FT,Data Analyst,45864,USD,45864,US,0,US,M


In [4]:
schema = StructType([
    StructField("work_year", IntegerType(), True),
    StructField("experience_level", StringType(), True),
    StructField("employment_type", StringType(), True),
    StructField("job_title", StringType(), True),
    StructField("salary", IntegerType(), True),
    StructField("salary_currency", StringType(), True),
    StructField("salary_in_usd", IntegerType(), True),
    StructField("employee_residence", StringType(), True),
    StructField("remote_ratio", IntegerType(), True),
    StructField("company_location", StringType(), True),
    StructField("company_size", StringType(), True)
])

In [5]:
salaries_df = spark.createDataFrame(data_engineer_salary.values.tolist(), schema=schema)

salaries_df.printSchema()

root
 |-- work_year: integer (nullable = true)
 |-- experience_level: string (nullable = true)
 |-- employment_type: string (nullable = true)
 |-- job_title: string (nullable = true)
 |-- salary: integer (nullable = true)
 |-- salary_currency: string (nullable = true)
 |-- salary_in_usd: integer (nullable = true)
 |-- employee_residence: string (nullable = true)
 |-- remote_ratio: integer (nullable = true)
 |-- company_location: string (nullable = true)
 |-- company_size: string (nullable = true)



In [10]:
rdd = salaries_df.rdd

In [16]:

def testing_operation(row):
    avg_salary_usa  = 60000
    profit_rate = row['salary'] * row['work_year']  / avg_salary_usa ** 2

RDD Without caching

In [18]:
start_time = time.time()
rdd.map(testing_operation).collect()
end_time = time.time()
print(f"profit_rate Calculation time: {end_time - start_time} seconds")


profit_rate Calculation time: 42.813623905181885 seconds


RDD + Cache

In [21]:
rdd.cache()
start_time = time.time()
rdd.map(testing_operation).collect()
end_time = time.time()
print(f"profit_rate Calculation time: {end_time - start_time} seconds")

profit_rate Calculation time: 19.934429168701172 seconds
